### Memory

In some applications like chatbots, it is important to remember previous interactions to keep the whole context of a conversation. Memory does provide you an easy way to handle this

In [16]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [17]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()

history.add_user_message("hi!")
history.add_ai_message("hello my friend!")
history.messages

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='hello my friend!', additional_kwargs={}, response_metadata={})]

In [18]:
from langchain.memory import ConversationBufferMemory

# ConversationBufferMemory is a memory that stores the conversation in a buffer.
# It is a simple memory that stores the conversation in a dictionary.

memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("hello my friend!")
memory.load_memory_variables({})

{'history': 'Human: hi!\nAI: hello my friend!'}

In [29]:
from langchain_openai import ChatOpenAI
from langchain.chains.conversation.base import ConversationChain


llm = ChatOpenAI(temperature=0)
conversation = ConversationChain(
    llm=llm, verbose=True, memory=ConversationBufferMemory()
)
conversation.invoke(input="What is the capital of France?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: What is the capital of France?
AI:

> Finished chain.


{'input': 'What is the capital of France?',
 'history': '',
 'response': 'The capital of France is Paris. It is known for its iconic landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral. Paris is also famous for its fashion, art, and cuisine.'}

In [20]:
conversation.invoke(input="Whats the best food there?")

{'input': 'Whats the best food there?',
 'history': 'Human: What is the capital of France?\nAI: The capital of France is Paris. It is known for its iconic landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral. Paris is also famous for its fashion, art, and cuisine.',
 'response': 'Some popular French dishes in Paris include croissants, escargot, coq au vin, and ratatouille. French cuisine is known for its use of fresh ingredients and rich flavors. Paris also has a wide variety of restaurants offering international cuisine from around the world.'}

memory=ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[])) verbose=True llm=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x118924b90>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x10f06f7d0>, root_client=<openai.OpenAI object at 0x1184133d0>, root_async_client=<openai.AsyncOpenAI object at 0x10f06bcd0>, temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********')) output_parser=StrOutputParser() llm_kwargs={}


### ConversationSummaryMemory

When inputs get long, we might not want to send the whole conversation, but rather a summary.

In [21]:
from langchain.memory import ConversationSummaryBufferMemory

review = "I ordered Pizza Salami for 9.99$ and it was awesome! \
The pizza was delivered on time and was still hot when I received it. \
The crust was thin and crispy, and the toppings were fresh and flavorful. \
The Salami was well-cooked and complemented the cheese perfectly. \
The price was reasonable and I believe I got my money's worth. \
Overall, I am very satisfied with my order and I would recommend this pizza place to others."

summary_memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100) # If Token is 1000, no summary will be created
summary_memory.save_context(
    {"input": "Hello, how can I help you today?"},
    {"output": "Could you analyze a review for me?"},
)
summary_memory.save_context(
    {"input": "Sure, I'd be happy to. Could you provide the review?"},
    {"output": f"{review}"},
)

In [22]:
conversation = ConversationChain(
    llm=llm, verbose=True, memory=summary_memory
)

conversation.invoke(input="Thank you very much!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human offers to help the AI and asks how they can assist. The AI requests an analysis of a review and the human agrees to help if the review is provided.
AI: I ordered Pizza Salami for 9.99$ and it was awesome! The pizza was delivered on time and was still hot when I received it. The crust was thin and crispy, and the toppings were fresh and flavorful. The Salami was well-cooked and complemented the cheese perfectly. The price was reasonable and I believe I got my money's worth. Overall, I am very satisfied with my order and I would recommend this pizza place to others.
Human: Thank you very much!
AI:

> Finished chain.


{'input': 'Thank you very much!',
 'history': "System: The human offers to help the AI and asks how they can assist. The AI requests an analysis of a review and the human agrees to help if the review is provided.\nAI: I ordered Pizza Salami for 9.99$ and it was awesome! The pizza was delivered on time and was still hot when I received it. The crust was thin and crispy, and the toppings were fresh and flavorful. The Salami was well-cooked and complemented the cheese perfectly. The price was reasonable and I believe I got my money's worth. Overall, I am very satisfied with my order and I would recommend this pizza place to others.",
 'response': "You're welcome! Now, could you please analyze this review for me?"}

In [23]:
memory.load_memory_variables({})

{'history': 'Human: hi!\nAI: hello my friend!'}

In [34]:
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains.conversation.base import ConversationChain

# Initialize the LLM
llm = ChatOpenAI(temperature=0)

# Create the summary memory with a token limit
summary_memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=150)

# Create a conversation chain
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=summary_memory
)

responses = []
responses.append(conversation.invoke(input="Hello! I'd like to get your opinion on a restaurant review I received."))
responses.append(conversation.invoke(input="Sure, I'd be happy to help! Could you share the review with me?"))
responses.append(conversation.invoke(input="""Here's the review: I ordered Pizza Salami for 9.99$ and it was awesome! 
The pizza was delivered on time and was still hot when I received it. 
The crust was thin and crispy, and the toppings were fresh and flavorful. 
The Salami was well-cooked and complemented the cheese perfectly. 
The price was reasonable and I believe I got my money's worth. 
Overall, I am very satisfied with my order and I would recommend this pizza place to others."""))
responses.append(conversation.invoke(input="What aspects of the review stood out to you?"))
responses.append(conversation.invoke(input="Do you think the price of 9.99$ is reasonable for this pizza?"))
responses.append(conversation.invoke(input="Would you recommend this restaurant based on the review?"))
responses.append(conversation.invoke(input="What other information would you have liked to see in the review?"))

for i, response in enumerate(responses, 1):
    print(f"\nExchange {i}:")
    print(f"Input: {response['input']}")
    print(f"Response: {response['response']}")
    print(f"History: {response['history']}")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hello! I'd like to get your opinion on a restaurant review I received.
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello! I'd like to get your opinion on a restaurant review I received.
AI: Hello! I'd be happy to help. Please provide me with the details of the restaurant review and I'll give you my opinion based on the information provided.
Human: Sure, I'd 